In [1]:
using SeisNoise, SeisIO, Plots, HDF5
using Dates

# Set station and channel dynamically
station1 = "NCBC"
station2 = "NCBC"
channel1 = "HHZ"
channel2 = "HHN"

fs = 40.0 # sampling frequency in Hz
freqmin,freqmax = 0.01,16 # min and max frequencies
cc_step, cc_len = 450, 1800 # corrleation step and length in S
maxlag = 100. # maximum lag time in correlation

# Load StationXML metadata once at the start
resp_file_1 = "/data/wsd02/maleen_data/NEPTUNE-Data/StationXML/NV.$(station1)..$(channel1).xml" 
resp_file_2 = "/data/wsd02/maleen_data/NEPTUNE-Data/StationXML/NV.$(station2)..$(channel2).xml" 
meta1 = SeisIO.read_meta("sxml",resp_file_1)
meta2 = SeisIO.read_meta("sxml",resp_file_2)

total_years = 2021 - 2016 + 1
total_days = 366
total_iterations = total_years * total_days
current_iteration = 0

for j in 2016:2021
    for i in 1:366
        # We have used string interpolation here to automatically insert station and channel values
        a = "/data/wsd02/maleen_data/NEPTUNE-Data/$(station1)/$(j)/$(lpad(i, 3, '0'))/$(station1).NV.$(j).$(lpad(i, 3, '0')).$(channel1)"
        b = "/data/wsd02/maleen_data/NEPTUNE-Data/$(station2)/$(j)/$(lpad(i, 3, '0'))/$(station2).NV.$(j).$(lpad(i, 3, '0')).$(channel2)"

        try
            S1 = SeisIO.read_data(a)
            S2 = SeisIO.read_data(b)
            S1.fs = [40.0]
            S2.fs = [40.0]
            SeisIO.translate_resp!(S1, meta1.resp[1], chans=1)
            SeisIO.translate_resp!(S2, meta1.resp[1], chans=1)
            SeisIO.remove_resp!(S1)
            SeisIO.remove_resp!(S2)

            process_raw!(S1, fs)
            process_raw!(S2, fs)


            R = RawData.([S1,S2], cc_len, cc_step)
            detrend!.(R)
            taper!.(R)
            bandpass!.(R, freqmin, freqmax, zerophase=true)
            clip!.(R,3)
            FFT = rfft.(R)
            whiten!.(FFT, freqmin, freqmax)
            C = correlate(FFT[1], FFT[2], maxlag)
            save_corr(C, "/data/wsd02/maleen_data/C_test/b/")
            # Update the iteration count
            current_iteration += 1

            # Calculate percentage
            percentage = (current_iteration / total_iterations) * 100

            # Print the percentage
            print("\rProgress: $(round(percentage, digits=2))%")
            flush(stdout)  # Make sure the print updates immediately

        catch
            current_iteration += 1
            #println("cannot process " * a)
        end
    end
end

Progress: 99.95%

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/home/seismic/.julia/packages/Plots/1KWPG/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x00007f55469bb47f, Ptr{Nothing} @0x00007f5546a5a70c, Ptr{Nothing} @0x00007f5546a5ae75, Ptr{Nothing} @0x00007f5546a5a90d, Ptr{Nothing} @0x00007f5546a5b6d1, Ptr{Nothing} @0x00007f5546a5c2f7, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x00007f5546a797a3, Ptr{Nothing} @0x00007f5546a79e3c, Ptr{Nothing} @0x00007f5546a799e4, Ptr{Nothing} @0x00007f5546a799e4, Ptr{Nothing} @0x00007f5546a7b629, Ptr{Nothing} @0x00007f55265ad83a, Ptr{Nothing} @0x00007f5546a3be09, Ptr{Nothing} @0x00007f552669d35a, Ptr{Nothing} @0x00007f55261bac15, Ptr{Nothing} @0x00007f5546a3be09, Ptr{Nothing} @0x00007f5546a4c5d6, Ptr{Nothing} @0x00007f55240f38aa, Ptr{Nothing} @0x00007f5546a3be09, Ptr{Nothing} @0x00007f5546a5ae95, Ptr{Nothing} @0x00007f55

In [ ]:
using SeisNoise, SeisIO, Plots, HDF5

# Set station and channel dynamically
station1 = "NC89"
station2 = "NC89"
channel1 = "HHZ"
channel2 = "HHE"

fs = 40.0 # sampling frequency in Hz
freqmin,freqmax = 0.01,16 # min and max frequencies
cc_step, cc_len = 450, 1800 # corrleation step and length in S
maxlag = 100. # maximum lag time in correlation

# Load StationXML metadata once at the start
resp_file_1 = "/data/wsd02/maleen_data/NEPTUNE-Data/StationXML/NV.$(station1)..$(channel1).xml" 
resp_file_2 = "/data/wsd02/maleen_data/NEPTUNE-Data/StationXML/NV.$(station2)..$(channel2).xml" 
meta1 = SeisIO.read_meta("sxml",resp_file_1)
meta2 = SeisIO.read_meta("sxml",resp_file_2)

for j in 2010:2021
    for i in 1:366
        # We have used string interpolation here to automatically insert station and channel values
        a = "/data/wsd02/maleen_data/NEPTUNE-Data/$(station1)/$(j)/$(lpad(i, 3, '0'))/$(station1).NV.$(j).$(lpad(i, 3, '0')).$(channel1)"
        b = "/data/wsd02/maleen_data/NEPTUNE-Data/$(station2)/$(j)/$(lpad(i, 3, '0'))/$(station2).NV.$(j).$(lpad(i, 3, '0')).$(channel2)"

        try
            S1 = SeisIO.read_data(a)
            S2 = SeisIO.read_data(b)
            S1.fs = [40.0]
            S2.fs = [40.0]
            SeisIO.translate_resp!(S1, meta1.resp[1], chans=1)
            SeisIO.translate_resp!(S2, meta1.resp[1], chans=1)
            SeisIO.remove_resp!(S1)
            SeisIO.remove_resp!(S2)

            process_raw!(S1, fs)
            process_raw!(S2, fs)


            R = RawData.([S1,S2], cc_len, cc_step)
            detrend!.(R)
            taper!.(R)
            bandpass!.(R, freqmin, freqmax, zerophase=true)
            clip!.(R,3)
            FFT = rfft.(R)
            whiten!.(FFT, freqmin, freqmax)
            C = correlate(FFT[1], FFT[2], maxlag)
            save_corr(C, "/data/wsd02/maleen_data/C_test/")
            print("$(i)_$(j),")

        catch
            println("cannot process " * a)
        end
    end
end